In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from Helper_Functions import Generate_Features, Quality_Evaluation
from skimage.io import imread
from skimage.transform import rescale
from sklearn.ensemble import RandomForestClassifier
from skimage.segmentation import random_walker
from multiprocessing import cpu_count
from skimage.morphology import opening, disk
import os
from sklearn.externals import joblib

In [ ]:
GT_path="/home/thanos/Desktop/Project_ADAS_2017/CITYSCAPE/Ground_Truth/train/strasbourg"
Im_path="/home/thanos/Desktop/Project_ADAS_2017/CITYSCAPE/Images/train/strasbourg"

def Get_Image_and_Ground_Truth(GT_path,Im_path):
    GT=[]
    Im=[]
    for file in os.listdir(GT_path):
        if file.endswith("_gtFine_labelIds.png"):
            GT.append(file)


    Im=os.listdir(Im_path)

    GT=sorted(GT)
    Im=sorted(Im)
    return GT,Im

In [ ]:
GT,Im=Get_Image_and_Ground_Truth(GT_path,Im_path)

Features=[]
Labels=[]
#0-173
scale_factor=0.25
steps=len(GT)
for i in range(steps):
    print('step = {}'.format(i))
    Train_GT=imread(GT_path+"/"+GT[i])
    Image_Train=imread(Im_path+"/"+Im[i])
    
    # Train Image
    I_train=rescale(Image_Train,scale_factor,mode='reflect')
    nx,ny=I_train[:,:,0].shape
    # Train Labels
    Train_Labels=rescale(Train_GT,scale_factor,mode='reflect', preserve_range=True)
    mask_Train = Train_Labels == 7
    # Find Train Features 
    Train_Features=np.reshape(Generate_Features(I_train),[54,nx*ny]).T
    Features.append(Train_Features)
    Labels.append(mask_Train.reshape(-1).astype('int'))
Features=np.array(Features)
Features=Features.reshape(steps*nx*ny,54)
Labels=np.array(Labels)
Labels=Labels.reshape(steps*nx*ny)    
print("Feature and Label generation completed")

In [ ]:
print(np.array(Features).shape)
print(np.array(Labels).shape)
RF = RandomForestClassifier(n_estimators=256,max_depth=20,n_jobs=1)
RF.fit(Features,Labels)

In [ ]:
joblib.dump(RF, 'RF_strasbourg_mxd20est256.pkl') 
